In [6]:
!curl localhost:9200

{
  "name" : "0bba196a300d",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "jjg01jbEThWhIc80sj6ebA",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [7]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

# Indexing the data

In [8]:
!pip install Elasticsearch

In [9]:
# Use Elastic search to index documents
from elasticsearch import Elasticsearch

es = Elasticsearch("http://127.0.0.1:9200")
es.info()

ObjectApiResponse({'name': '0bba196a300d', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'jjg01jbEThWhIc80sj6ebA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [10]:
# Create an index (in elastic search its a table)
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
# Delete index if it already exists
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)
response = es.indices.create(index=index_name, body=index_settings)

response

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

Which function do you use for adding your data to elastic?

In [11]:
!pip install tqdm

In [12]:
#Add data to the table we craeted
from tqdm.auto import tqdm
for doc in tqdm(documents):
    es.index(index=index_name,document=doc)

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 948/948 [00:33<00:00, 28.36it/s]


# Q3. Searching

In [13]:
user_question = "How do I execute a command in a running docker container?"

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": user_question,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
        }
    }
}

In [14]:
answer=  es.search(index=index_name,body=search_query)
answer['hits']['hits'][0]["_score"]

84.050095

What is the score for the top ranking results?
**84.05**

# Q4. Filtering

In [15]:
filtered_user_question = "How do I execute a command in a running docker container?"

filtered_search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": filtered_user_question,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

In [16]:
filtered_answer = es.search(index=index_name,body=filtered_search_query)
filtered_answer['hits']['hits'][-1]["_source"]['question']

'How do I copy files from a different folder into docker container’s working directory?'

# Q5 Build a prompt

In [19]:
# Context template
context_template = """
Q: {question}
A: {text}
""".strip()

In [25]:
context_template

'Q: {question}\nA: {text}'

In [21]:
filtered_answer

ObjectApiResponse({'took': 33, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 345, 'relation': 'eq'}, 'max_score': 84.050095, 'hits': [{'_index': 'course-questions', '_id': '_933TZAB4alzajaLI6Nz', '_score': 84.050095, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'Ht33TZAB4alzajaLJ6Sh', '_score': 51.04628, '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo 

In [23]:
filtered_answer['hits']['hits']

[{'_index': 'course-questions',
  '_id': '_933TZAB4alzajaLI6Nz',
  '_score': 84.050095,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'Ht33TZAB4alzajaLJ6Sh',
  '_score': 51.04628,
  '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_

In [28]:
questions = [items["_source"]["question"] for items in filtered_answer['hits']['hits']]
questions
answers = [items["_source"]["text"] for items in filtered_answer['hits']['hits']]
answers

context = ""
for items in range(len(questions)):
    qn_ans = context_template.format(question=questions[items],text=answers[items])
    context+= ("\n\n"+ qn_ans)
context = context.strip()    

In [30]:
print(context)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [39]:
# Create our prompt
user_question = "How do I execute a command in a running docker container?"
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

prompt=prompt_template.format(question=user_question,context=context)

In [42]:
print(len(prompt))

1462


# Q6.Tokens

In [43]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 824.9 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.1/775.1 kB 14.5 MB/s eta 0:00:0000:01


In [45]:
import tiktoken

In [49]:
encoding = tiktoken.encoding_for_model("gpt-4o")
len(encoding.encode(prompt))

322

In [50]:
encoding.decode_single_token_bytes(63842)

b"You're"

In [55]:
# !pip install dotenv 

  Using cached dotenv-0.0.5.tar.gz (2.4 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... error
  error: subprocess-exited-with-error
  
  × pip subprocess to install backend dependencies did not run successfully.
  │ exit code: 1
  ╰─> [29 lines of output]
        Using cached distribute-0.7.3.zip (145 kB)
        Installing build dependencies: started
        Installing build dependencies: finished with status 'done'
        Getting requirements to build wheel: started
        Getting requirements to build wheel: finished with status 'done'
        Preparing metadata (pyproject.toml): started
        Preparing metadata (pyproject.toml): finished with status 'error'
        error: subprocess-exited-with-error
      
        × Preparing metadata (pyproject.toml) did not run successfully.
        │ exit code: 1
        ╰─> [6 lines of output]
            usage: setup.py [global_opts] cmd1 [cmd1_opts] [cmd2 [

In [2]:
from openai import OpenAI

In [ ]:
# from dotenv import load_dotenv

In [ ]:
# load_dotenv()

In [3]:
# Create a client object
client = OpenAI()
response = client.chat.completions.create(
    model="gpt-4o",
    messages = [{
        "role":"user",
        "content":prompt
    }])
print(response.choices[0].message.content)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable